In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import KNNImputer
from fuzzywuzzy import process

In [2]:
# Data
R = pd.read_csv('../data/UserRatingTitles-withoutYear.csv', index_col=0)

# KNN
imputer = KNNImputer(n_neighbors=5)
R = pd.DataFrame(imputer.fit_transform(R), columns=R.columns, index=R.index)


In [3]:
# cosine_matrix
similarities = pd.DataFrame(cosine_similarity(
    R), columns=R.index, index=R.index)

In [4]:
def convert_flask_dict(flask_dict):
    new_keys = list(flask_dict.values())[::2]
    new_vals = list(flask_dict.values())[1::2]

    return dict(zip(new_keys, new_vals))

In [6]:
user_input = {'Toy Story': 5, 'Sabrina': 1, 'My Man Godfrey': 2}

In [12]:



# function working for user_input = {'Toy Story': 5, 'Sabrina': 1, 'My Man Godfrey': 2}
def get_recommendations_cosine(user_input):
    flask_user_input = user_input

    # Introduce New User
    #new_user_df = pd.DataFrame(flask_user_input, index=['NewUser'])
    new_user_df = pd.DataFrame(
        [np.nan]*len(R.columns), index=R.columns)
    new_user_df.columns = ['NewUser']
    new_user_df=new_user_df.transpose()
    # Loop to check whether user_id is there
    for key, value in flask_user_input.items():
        if key in new_user_df.columns:
            new_user_df.loc[:, key] = float(value)
        else:
            closest_match = process.extract(key, R.columns)[0][0]
            new_user_df.loc[:, closest_match] = float(value)
            if len(process.extract(key, R.columns)[0][0]) < 0.5*len(key):
                closest_match = process.extract(key, R.columns)[1][0]
                new_user_df.loc[:, closest_match] = float(value)

    # Append newuser to the user-item matrix
    R_new_user = R.append(new_user_df)
    print(R_new_user)
    # Fill the NaNs for Paul
    R_new_user_filled = R_new_user.fillna(2.5)
    print(R_new_user_filled)
    # Create a filter for the missing movies
    movie_filter = ~R_new_user.isna().any().values
    print(movie_filter)
    # Create an updated user list
    updated_users = R_new_user.index
    print(updated_users)
    R_new_user.transpose()[movie_filter].transpose()

    # calculate a similarity to the other users based on dummys rating
    similarities_new_user = pd.DataFrame(cosine_similarity(R_new_user.transpose(
    )[movie_filter].transpose()), index=updated_users, columns=updated_users)
    print(similarities_new_user)
    # Predict ratings for Dummy

    similarities_new_user = similarities_new_user['NewUser'][~(
        similarities_new_user.index == 'NewUser')]

    # Calculate rating predictions
    rating_predictions = pd.DataFrame(
        np.dot(similarities_new_user, R)/similarities_new_user.sum(), index=R.columns)

    # Get recommendations
    #rating_predictions[~movie_filter].sort_values(by=0, ascending=False)
    films_recommended = rating_predictions[~movie_filter].sort_values(
        by=0, ascending=False).index[:3]

    return films_recommended


In [13]:
get_recommendations_cosine(user_input)

         Toy Story  Jumanji  Grumpier Old Men  Waiting to Exhale  \
NewUser        NaN      NaN               NaN                NaN   

         Father of the Bride Part II  Heat  Sabrina  Tom and Huck  \
NewUser                          NaN   NaN      NaN           NaN   

         Sudden Death  GoldenEye  ...  Gintama: The Movie  \
NewUser           NaN        NaN  ...                 NaN   

         anohana: The Flower We Saw That Day - The Movie  Silver Spoon  \
NewUser                                              NaN           NaN   

         Love Live! The School Idol Movie  Jon Stewart Has Left the Building  \
NewUser                               NaN                                NaN   

         Black Butler: Book of the Atlantic  No Game No Life: Zero  Flint  \
NewUser                                 NaN                    NaN    NaN   

         Bungo Stray Dogs: Dead Apple  Andrew Dice Clay: Dice Rules  
NewUser                           NaN                           Na

                                    Toy Story  Jumanji  Grumpier Old Men  \
1                                         4.0      3.6               4.0   
2                                         4.0      3.6               2.9   
3                                         3.7      3.0               2.1   
4                                         3.9      3.6               3.6   
5                                         4.0      3.4               3.3   
...                                       ...      ...               ...   
Black Butler: Book of the Atlantic        5.0      2.5               2.5   
No Game No Life: Zero                     5.0      2.5               2.5   
Flint                                     5.0      2.5               2.5   
Bungo Stray Dogs: Dead Apple              5.0      2.5               2.5   
Andrew Dice Clay: Dice Rules              5.0      2.5               2.5   

                                    Waiting to Exhale  \
1                             

                                           1         2         3         4  \
1                                   1.000000  0.998424  0.995780  0.998213   
2                                   0.998424  1.000000  0.998967  0.993340   
3                                   0.995780  0.998967  1.000000  0.989386   
4                                   0.998213  0.993340  0.989386  1.000000   
5                                   1.000000  0.998424  0.995780  0.998213   
...                                      ...       ...       ...       ...   
Black Butler: Book of the Atlantic  0.882900  0.898562  0.892337  0.859681   
No Game No Life: Zero               0.882900  0.898562  0.892337  0.859681   
Flint                               0.882900  0.898562  0.892337  0.859681   
Bungo Stray Dogs: Dead Apple        0.882900  0.898562  0.892337  0.859681   
Andrew Dice Clay: Dice Rules        0.882900  0.898562  0.892337  0.859681   

                                           5         6         

KeyError: 'NewUser'